# Property Information Web Scraping

In [23]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests
import re
import pandas as pd

In [24]:
domain  = ["19++E++04590", "31057++00040", "48161++00760", "27067++00060", "32420++01250"]

In [25]:
def county_data(domain):
    html = requests.get("{}{}{}".format("government website for the property information",domain,"/"))
    soup = bs(html.text, "html.parser")
    
    ###################### main page ########################
    # street address - main page
    # SBL code (Section/Block/Lot) - main page
    address = soup.find("div", class_="col-md-12").text.strip()
    sbl_code01 = soup.findAll("div", class_="col-md-4")[:3]
    main_dict={}
    main_dict['street adress'] = address
    # for i in range(len(sbd_code01)):
    sbl_colname = ['Section', "Block", "Lot"]
    for i in range(len(sbl_code01)):
        a = sbl_code01[i].text.strip()
        main_dict[sbl_colname[i]] = a
        
    ###################### value ########################
    # Fair market values
    # Assessed values 
    value1 = soup.find("div", class_="tab-pane", id="infovaltab")
    value2 = value1.find("div", class_="table-responsive")
    # value2
    Fair_value = {"Fair value 2020":value2.findAll('tr')[1].findAll('td')[4].text.strip()}
    Assess_value = {"Assessed value 2020":value2.findAll('tr')[4].findAll('td')[4].text.strip()}
    
    ###################### Property description ########################
    # Land category
    # Acres
    # Lot Square Footage (SQFT)
    # Year built
    # HEAT/AC
    # Total Living Area (SQFT)
    # Full Bathrooms
    # Number of Bedrooms
    # Basement Area
    # Attached Garage
    desc1 = soup.find("div", class_="tab-pane", id="infodesctab")
    desc2 = desc1.find("div", class_="tab-pane", id="protop")
    desc3 = desc2.find("div", class_="tab-content")
    Land_category = {"Land category": desc3.find("div", class_="tab-pane", id="prt2020").findAll("td")[1].text.strip()}


    
    desc2_1 = desc1.findAll("table", class_="table table-striped table-responsive")
    Pro_description = {}
    desc2_1[0].findAll('tr')
    for i in range(len(desc2_1)):
        if desc2_1[i].find('th').text.strip()=='Card':
            pass
        else:
            for t in range(len(desc2_1[i].findAll('tr'))):
                Pro_description[desc2_1[i].findAll('tr')[t].find('th').text.strip()] = desc2_1[i].findAll('tr')[t].find('td').text.strip()

    Pro_description_01 = {'Acres':"", 'Lot Square Footage (SQFT)':"", 'Year Built':"", 'Heat/AC':"", 'Total Living Area (SQFT)':"", 
                   'Full Bathrooms':"", 'Number of Bedrooms':"", 'Basement Area':"", 'Attached Garage':""}

    for key1, values1 in Pro_description_01.items():
        for key2, values2 in Pro_description.items():
            if key1 == key2:
                Pro_description_01[key1] = values2
    ####### Recent Sales ############
    sales1 = soup.find("div", class_="tab-pane", id="infosalestab")
    sales2 = sales1.find("table", class_="table table-striped")
    sales3 = sales2.findAll('tr')
    sale_info = [0,1,4]
    sales_dict = {}
    if len(sales3)==1:
        sales_dict["{}".format(sales3[0].findAll('th')[0].text.strip())] = "."
        sales_dict["{}".format(sales3[0].findAll('th')[1].text.strip())] = "."
        sales_dict["{}".format(sales3[0].findAll('th')[4].text.strip())] = "."
    else:
        for i in range(len(sales3)-1):
            aa = sales3[i+1].findAll('td')[0].text.strip()
            bb = sales3[i+1].findAll('td')[1].text.strip()
            cc = sales3[i+1].findAll('td')[4].text.strip()
            sales_dict["{} {}".format(sales3[0].findAll('th')[0].text.strip(), i+1)] = aa
            sales_dict["{} {}".format(sales3[0].findAll('th')[1].text.strip(), i+1)] = bb
            sales_dict["{} {}".format(sales3[0].findAll('th')[4].text.strip(), i+1)] = cc
    
    dataframe_list = []
    dataframe_nassau = pd.DataFrame.from_dict(main_dict, orient='index',columns=['{}'.format(domain)]).append(
                        pd.DataFrame.from_dict(Fair_value, orient='index',columns=['{}'.format(domain)])).append(
                        pd.DataFrame.from_dict(Assess_value, orient='index',columns=['{}'.format(domain)])).append(
                        pd.DataFrame.from_dict(Land_category, orient='index',columns=['{}'.format(domain)])).append(
                        pd.DataFrame.from_dict(Pro_description_01, orient='index',columns=['{}'.format(domain)])).append(
                        pd.DataFrame.from_dict(sales_dict, orient='index',columns=['{}'.format(domain)]))

            
    return(dataframe_nassau.T)

In [26]:
county_data(domain[1])

,street adress,Section,Block,Lot,Fair value 2020,Assessed value 2020,Land category,Acres,Lot Square Footage (SQFT),Year Built,Heat/AC,Total Living Area (SQFT),Full Bathrooms,Number of Bedrooms,Basement Area,Attached Garage,Date of Sale,Sale Price,Condition of Transfer
31057++00040,"Address: 61 DOSORIS LN. GLEN COVE, 11542",Section: 31,Block: 057,Lot: 4,"$369,000",369,RESIDENTIAL,.1616,7040,1948,CNTRL HEAT,1488,1,,FULL,No,.,.,.
